# 非線形最適化

目的関数が1変数関数のときの最急降下法を実装してみる。

In [31]:
def backTrack(f, df, x, d, alpha=10, rho=0.5, c1=0.01):
    while(f(x + alpha*d) > f(x) + c1 * df(x) * d * alpha):
        alpha = rho*alpha
    return alpha

def grad(f, x0, df=None, eps=0.001):
    '''
    Parameters
    ----------
    f: objective function
    x0: initial point
    df: derivative
    eps: epsilon
    
    Return
    ------
    x^*: minimizer
    '''
    while(abs(df(x0)) > eps):
        if df == None:
            df = lambda x0: (f(x0)+f(x0+0.01))/0.01
        d = -df(x0)
        alpha = backTrack(f, df, x0, d)
        x0 = x0 + alpha*d
    return x0

In [6]:
f = lambda x: x**2

In [3]:
f(2)

4

In [32]:
grad(f, 100, df=lambda x: 2*x)

-0.0003814697265625

In [33]:
import numpy as np
from scipy.optimize import minimize, newton

In [38]:
minimize(f,100)

      fun: 5.552074997367714e-17
 hess_inv: array([[ 0.50000004]])
      jac: array([ -1.28826571e-12])
  message: 'Optimization terminated successfully.'
     nfev: 21
      nit: 4
     njev: 7
   status: 0
  success: True
        x: array([ -7.45122473e-09])

## 2変数のも作ってみる

In [40]:
g = lambda x, y: x + 2*y

In [41]:
g(1,2)

5

In [1]:
def backTrack(f, df, x, d, alpha=10, rho=0.5, c1=0.01):
    while(f(x + alpha*d) > f(x) + c1 * np.dot(df(x), d) * alpha):
        alpha = rho*alpha
    return alpha

def grad(f, x0, df=None, eps=0.001):
    '''
    Parameters
    ----------
    f: objective function
    x0: initial point
    df: derivative
    eps: epsilon
    
    Return
    ------
    x^*: minimizer
    '''
    while(np.linalg.norm(df(x0)) > eps):
        d = -df(x0)
        alpha = backTrack(f, df, x0, d)
        x0 = x0 + alpha*d
    return x0

In [2]:
import numpy as np
l = 10
def f(z):
    x = z[0]
    y = z[1]
    val = - x - y - l * (np.log((-1/3)*x-y+1) + np.log(-3*x-y+3) + np.log(x) + np.log(y) )
    return val

def df(z):
    x = z[0]
    y = z[1]
    val = [ (1 + l * (-1/(-x-3*y+3) - 3/(-3*x-y+3) + 1/x)), (1 + l * (-3/-x-3*y+3 - 1/(-3*x-y+3) + 1/y)) ]
    return np.array(val)

In [3]:
f([0.5,0.5])

23.849066497880003

In [4]:
df([0.5,0.5])

array([-19.,  86.])

In [6]:
x0 = np.array([0.5, 0.5])
grad(f, df, x0)

TypeError: 'numpy.ndarray' object is not callable

# Implementing Simplex Method

In [101]:
A = np.array([
        [1.0,1,2,1,0,0],
        [2,0,2,0,1,0],
        [2,1,3,0,0,1]
    ])
c = np.array([3.0,2,4,0,0,0])
b = np.array([4.0,5,7])
basis = [3,4,5]
nonbasis = [0,1,2]

n = len(c)
m = len(A)

In [102]:
c_nonbasis = np.array([c[i] if i in nonbasis else -float('inf') for i in range(len(c))])
j_pivot = np.argmax(c_nonbasis)
j_pivot

2

In [103]:
i_pivot = np.argmin(b / A.T[j_pivot])
i_pivot

0

In [104]:
stepsize = np.min(b / A.T[j_pivot])
stepsize

2.0

In [105]:
stepsize * c[j_pivot]

8.0

In [106]:
# update basis and nonbasis
nonbasis.append(basis[i_pivot])
nonbasis.remove(j_pivot)
basis[i_pivot] = j_pivot

In [107]:
basis

[2, 4, 5]

In [108]:
nonbasis

[0, 1, 3]

In [109]:
b

array([ 4.,  5.,  7.])

In [110]:
A

array([[ 1.,  1.,  2.,  1.,  0.,  0.],
       [ 2.,  0.,  2.,  0.,  1.,  0.],
       [ 2.,  1.,  3.,  0.,  0.,  1.]])

In [111]:
b[i_pivot], A[i_pivot] = b[i_pivot] / A[i_pivot][j_pivot], A[i_pivot] / A[i_pivot][j_pivot]

In [91]:
b

array([ 2.,  5.,  7.])

In [92]:
A

array([[ 0.5,  0.5,  1. ,  0.5,  0. ,  0. ],
       [ 2. ,  0. ,  2. ,  0. ,  1. ,  0. ],
       [ 2. ,  1. ,  3. ,  0. ,  0. ,  1. ]])

In [112]:
A[i_pivot][j_pivot] = 0

In [94]:
A

array([[ 0.5,  0.5,  0. ,  0.5,  0. ,  0. ],
       [ 2. ,  0. ,  2. ,  0. ,  1. ,  0. ],
       [ 2. ,  1. ,  3. ,  0. ,  0. ,  1. ]])

In [113]:
for i in range(m):
    if i != i_pivot:
        A[i] = A[i] - A[i_pivot] * A[i][j_pivot]
        b[i] = b[i] - b[i_pivot] * A[i][j_pivot]
        A[i][j_pivot] = 0

In [114]:
A

array([[ 0.5,  0.5,  0. ,  0.5,  0. ,  0. ],
       [ 1. , -1. ,  0. , -1. ,  1. ,  0. ],
       [ 0.5, -0.5,  0. , -1.5,  0. ,  1. ]])

In [97]:
b

array([ 2.,  1.,  1.])

In [117]:
c_new = c - A[i_pivot] * c[j_pivot]
c_new[j_pivot] = 0
c_new

array([ 1.,  0.,  0., -2.,  0.,  0.])

In [118]:
A

array([[ 0.5,  0.5,  0. ,  0.5,  0. ,  0. ],
       [ 1. , -1. ,  0. , -1. ,  1. ,  0. ],
       [ 0.5, -0.5,  0. , -1.5,  0. ,  1. ]])

In [119]:
b

array([ 2.,  1.,  1.])

In [120]:
c

array([ 3.,  2.,  4.,  0.,  0.,  0.])

In [122]:
basis

[2, 4, 5]

In [123]:
nonbasis

[0, 1, 3]

In [121]:
c_nonbasis = np.array([c[i] if i in nonbasis else -float('inf') for i in range(len(c))])
c_nonbasis

array([  3.,   2., -inf,   0., -inf, -inf])

In [125]:
j_pivot = np.argmax(c_nonbasis)

In [126]:
np.argmin(b / A.T[j_pivot])

1

In [145]:
import numpy as np


def simplex(c, A, b, basis, nonbasis, x=None):
    '''
    c: n x 1, A: m x n, b = m x 1
    x: n x 1, initial value
    basis: m x 1, nonbasis: (n-m) x 1

    c, A, b, x: numpy array
    basis, nonbasis: python list
    '''

    m = len(A)
    n = len(c)

    if x is None:
        x = np.array([0.0] * n)

    optVal = x.dot(c)

    # check feasibility
    for test in range(m):
        if (A.dot(x) > b)[test]:
            print('infeasible')
            return None

    # pivoting
    EPSILON = 10**(-5)
    TIME = 0
    while(np.max(c) > EPSILON):
        # choose j_pivot
        c_nonbasis = np.array([c[i] if i in nonbasis else -float('inf') for i in range(len(c))])
        j_pivot = np.argmax(c_nonbasis)

        # check boundedness
        a = np.max(A.T[j_pivot])
        if a <= 0:
            print('unbounded')
            return None

        # choose pivot
        pivot_column = [i / j if j != 0 else float('inf') for i, j in zip(b, A.T[j_pivot])]
        # pivot_column = b / A.T[j_pivot] としてもよいがdevided by zero errorがうるさい
        pivot_column = [pivot_column[i] if pivot_column[i] >= 0 else float('inf') for i in range(len(pivot_column))]
        i_pivot = np.argmin(pivot_column)
        stepsize = np.min(pivot_column)
        optVal += stepsize * c[j_pivot]

        # update basis and nonbasis
        nonbasis.append(basis[i_pivot])
        nonbasis.remove(j_pivot)
        basis[i_pivot] = j_pivot

        # update A, b, and c
        b[i_pivot], A[i_pivot] = b[i_pivot] / A[i_pivot][j_pivot], A[i_pivot] / A[i_pivot][j_pivot]
        A[i_pivot][j_pivot] = 0

        for i in range(m):
            if i != i_pivot:
                A[i] = A[i] - A[i_pivot] * A[i][j_pivot]
                b[i] = b[i] - b[i_pivot] * A[i][j_pivot]
                A[i][j_pivot] = 0

        c = c - A[i_pivot] * c[j_pivot]
        c[j_pivot] = 0

        # for debugging
        TIME += 1
        solution = [b[basis.index(i)] if i in basis else 0 for i in range(n)]
        print('loop no.{}: val = {}、sol = {}'.format(TIME, optVal, solution))
        if TIME > 100:
            break


    print('opt.val. = {}, opt.sol. = {}'.format(optVal, solution))



In [146]:
A = np.array([
        [1.0,1,2,1,0,0],
        [2,0,2,0,1,0],
        [2,1,3,0,0,1]
    ])
c = np.array([3.0,2,4,0,0,0])
b = np.array([4.0,5,7])
basis = [3,4,5]
nonbasis = [0,1,2]

In [147]:
simplex(c,A,b,basis,nonbasis)

loop no.1: val = 8.0、sol = [0, 0, 2.0, 0, 1.0, 1.0]
loop no.2: val = 9.0、sol = [1.0, 0, 1.5, 0, 0, 0.5]
loop no.3: val = 10.5、sol = [2.5, 1.5, 0, 0, 0, 0.5]
opt.val. = 10.5, opt.sol. = [2.5, 1.5, 0, 0, 0, 0.5]


In [10]:
A

array([[ 1.,  1.,  2.,  1.,  0.,  0.],
       [ 0., -2., -2., -2.,  1.,  0.],
       [ 0., -1., -1., -2.,  0.,  1.]])

In [9]:
c

array([ 3.,  2.,  4.,  0.,  0.,  0.])

In [132]:
test = np.array([2,-1,0,3])
test_ = [test[i] if test[i] >= 0 else float('inf') for i in range(len(test))]

In [133]:
test_

[2, inf, 0, 3]

In [136]:
test = np.array([4,5,7])
test0 = np.array([2,0,3])
test1 = [i/j if j != 0 else float('inf') for i,j in zip(test, test0)]

In [137]:
test1

[2.0, inf, 2.3333333333333335]

In [144]:
basis = [2,3,5]
b = [5,3,4]
sol = [b[basis.index(i)] if i in basis else 0 for i in range(6)]
sol

[0, 0, 5, 3, 0, 4]